In [19]:
from time import sleep
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
RESULTS_FILE = 'nissan_info.xlsx'
BASE_URL = 'https://auto.drom.ru/nissan/used/all'

In [20]:
def parse_auto_page(page_url: str) -> dict[str, str]:
    req = requests.get(page_url)
    parsed = BeautifulSoup(req.content, 'html.parser')
    all_tables = parsed.findAll('table')
    if not all_tables:
        return {}
    car_info = all_tables[0]
    data = {part_info.find('th').text: part_info.find('td').text.replace('\xa0', ' ') for part_info in car_info.findAll('tr') if part_info.find('th')}
    data['page_title'] = [s.text for s in parsed.select('h1 > span')][0]
    data['price'] = [s.text.replace('\xa0', ' ') for s in parsed.select('.wb9m8q0')][0]
    pub_date = parsed.select('.css-pxeubi.evnwjo70')
    if pub_date:
        dates = re.findall(r'\d\d\.\d\d.\d\d\d\d', pub_date[0].text)
        if dates:
            data['publication_date'] = dates[0]
    return data

In [5]:
def get_all_links(page_url: str) -> list[dict[str, str]]:
    r = requests.get(page_url)
    r.raise_for_status()
    parsed_list = BeautifulSoup(r.content, 'html.parser')
    car_data = []
    for car_page in parsed_list.select('a>h3'):
        car_info = {
            'title': car_page.text,
            'link': car_page.parent.attrs['href']
        }
        car_data.append(car_info)
    return car_data



# try:
#     car_info = parse_auto_page(link)
# except Exception as e:
#     print(f'{link}\n\t{r.status_code=}\n\t{e}')
#     continue 

In [27]:
def get_all_model_links(base_url: str, max_page: int = 100) -> list[dict[str, str]]:
    car_links = []
    for i in tqdm(range(1, max_page)):
        parsed = get_all_links(base_url + f'/page{i}/')
        if len(parsed) == 0:
            print('No more cars to parse')
            break
        sleep(1)
        car_links.extend(parsed)
    return car_links

In [28]:
car_links = get_all_model_links(BASE_URL, 100)
car_info = []
for car_link in tqdm(car_links):
    car_link.update(parse_auto_page(car_link))
    sleep(1)

  0%|          | 0/99 [00:00<?, ?it/s]

KeyboardInterrupt: 

ValueError: Invalid isoformat string: '05.07.2024'

In [15]:
import re

In [25]:
from datetime import datetime

In [11]:
car_links[0]['link']

'https://auto.drom.ru/moscow/nissan/x-trail/921373117.html'